# Strategy-V1

We aim to build a strategy around the following instruments: 0, 1, 2, 3, 4, 5, 6, 7, 22, 23, 24, 
26, 27

From our analysis, we gathered that the set of instruments followed trending/momentum behaviour. 
Therefore, we outline the key characteristics of our strategy below:

- **Features to Develop**
    - EMA Crossovers with a short EMA (5-10 day lookback) and long EMA (50-200 day lookback) OR 
        Moving Average Convergence Divergence (MACD) - lookbacks to be grid searched
    - Rolling Auto-Correlation
    - Relative Strength Index (lookback of 14 days)
    - Rolling Volatility (lookback to be grid searched)
    - Bollinger Bands - middle band lookback to be grid-searched, upper-band and lower band 
        volatility multiplier to be grid-searched
- **Strategies to Explore**:
    - Time-Series Momentum: does this asset perform well relative to its own past?
    - Moving Average Crossover System: for e.g. when the short EMA goes above the long EMA, its a
         bullish signal
    - Mean Reversion (Sparingly): only in periods where auto-correlation is negative

We do an initial setup below

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from typing import List, Tuple

from numpy import ndarray
from pandas import Series, DataFrame

instrument_nos: List[int] = [0, 1, 2, 3, 4, 5, 6, 7, 22, 23 ,24, 26, 27]
prices_filepath: str = "../prices.txt"

## Feature Engineering

### 1. Exponential Moving Average

In [ ]:
def get_ema(instrument_price_history: Series, lookback: int) -> float:
    if len(instrument_price_history) < lookback: return 0
    
    return instrument_price_history.ewm(span=lookback, adjust=False).mean()

### 2. Rolling Auto-Correlation

In [ ]:
def get_rolling_autocorrelation(instrument_price_history: Series, lag_limit: int, lookback: int) \
        -> ndarray:
     if len(instrument_price_history) < lookback: return np.zeros(lag_limit)
     
     prices_window: ndarray = instrument_price_history.to_numpy()[-lookback:]
     
     mean: float = prices_window.mean()
     demeaned = prices_window - mean
     denominator: float = np.dot(demeaned, demeaned)
     autocorrelations: ndarray = np.array([np.dot(demeaned[k:], demeaned[:-k]) / denominator
           for k in range(1, lag_limit + 1)])
     
     return autocorrelations

### 3. Relative Strength Index

In [ ]:
def get_rsi(instrument_price_history: Series, lookback: int = 14) -> float:
    prices_window: ndarray = instrument_price_history.to_numpy()[-lookback:]
    delta_prices: ndarray = np.diff(prices_window)
    
    gains: ndarray = np.where(delta_prices > 0, delta_prices, 0)
    losses: ndarray = np.where(delta_prices < 0, -delta_prices, 0)
    
    average_gain: float = gains.mean()
    average_loss: float = losses.mean()
    
    if average_loss == 0: return 100.0
    
    relative_strength: float = average_gain / average_loss
    relative_strength_index: float = 100 - (100 / (1 + relative_strength))
    return relative_strength_index

### 4. Annualised Rolling Volatility

In [ ]:
def get_annualised_volatility(instrument_price_history: Series, lookback: int) -> float:
    if len(instrument_price_history) < lookback: return 0.0
    
    prices_window: Series= instrument_price_history[-lookback:]
    returns: Series = prices_window.pct_change().fillna(0)
    standard_deviation: float = returns.rolling(window=lookback).std()
    return standard_deviation * (252 ** 0.5)

### 5. Bollinger Bands

In [ ]:
def get_bollinger_bands(instrument_price_history: Series, lookback: int, width_multiplier: float) \
        -> ndarray:
    if len(instrument_price_history) < lookback: return np.zeros(3)
    
    prices_window: Series = instrument_price_history[-lookback:]
    
    middle_band: float = prices_window.mean()
    
    standard_deviation: float = get_annualised_volatility(instrument_price_history, lookback) /(252 
        ** 0.5)
    
    lower_band: float = middle_band - width_multiplier * standard_deviation
    upper_band: float = middle_band + width_multiplier * standard_deviation
    
    bollinger_bands: ndarray = np.array([lower_band, middle_band, upper_band], dtype=float)
    return bollinger_bands
    

### 6. Moving Average Convergence Divergence

In [ ]:
def get_macd(instrument_price_history: Series, fast_lookback: int, slow_lookback: int,
    signal_period: int) -> Tuple[
    float, float]:
    if len(instrument_price_history) < fast_lookback: return 0,0
    
    ema_fast: Series = instrument_price_history.ewm(span=fast_lookback, adjust=False).mean()
    ema_slow: Series = instrument_price_history.ewm(span=slow_lookback, adjust=False).mean()
    
    macd_line = ema_fast - ema_slow
    
    signal_line = macd_line.ewm(span=signal_period, adjust=False).mean()
    
    macd_today: float = float(macd_line.iloc[-1])
    signal_today: float = float(signal_line.iloc[-1])
    
    return macd_today, signal_today

## Strategy Implementation

### Version 1

- Simple EMA crossover
    - If short EMA is above long EMA, buy
    - If short EMA is below long EMA, sell

In [ ]:
from backtester import Backtester, BacktesterResults, Params

positions_limit: int = 10000
def getMyPosition(prices_so_far: ndarray, allocated_instruments: List[int]) -> ndarray:
    positions: ndarray = np.zeros(50)
    
    short_ema_lookback: int = 5
    long_ema_lookback: int = 50
    
    if len(prices_so_far.iloc[0]) < long_ema_lookback:
        return positions
    
    for instrument_no in allocated_instruments:
        instrument_price_data: Series = prices_so_far.iloc[instrument_no]
        
        short_ema: float = get_ema(instrument_price_data, short_ema_lookback)
        long_ema: float = get_ema(instrument_price_data, long_ema_lookback)
        
        if short_ema > long_ema:
            positions[instrument_no] = positions_limit / instrument_price_data[-1]
        elif short_ema < long_ema:
            positions[instrument_no] = -(positions_limit / instrument_price_data[-1])
        
    return positions
